In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import guild.ipy as guild
import matplotlib.pyplot as plt
from matplotlib import transforms
import plotly.express as px
import seaborn as sns
from guild import tfevent, config
from itertools import chain

# config.set_guild_home("/home/miniconda3/envs/envname/.guild")
guild_runs = guild.runs()
flags = guild_runs.flags()
scalars = guild_runs.scalars()
HOME = config.guild_home()
orig_runs = guild_runs.compare()
sns.set(font_scale=2)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from datetime import datetime, date, timedelta
# today_mask = runs["started"].apply(lambda d: d.date()) >= date.today()
ndays_ago = 40
nhours_ago = 5
# today_mask = runs["started"].apply(lambda d: d.date()) >= date.today() - timedelta(days=ndays_ago, hours=nhours_ago)
today_mask = orig_runs["started"] >= datetime.now() - timedelta(days=ndays_ago, hours=nhours_ago)
no_errors_mask = orig_runs["status"] != "error"
imputer_task_mask = orig_runs["operation"] == "imputer"

# I want to ignore the previous APs and just use the current one because my fixes are in the current one
# tmp_ignore = (orig_runs["started"] < datetime.now() - timedelta(days=1)) & (orig_runs["label"] == "ap ckd")

mask = today_mask & no_errors_mask & imputer_task_mask # & ~tmp_ignore
runs = orig_runs[mask]


flags_mask = flags["run"].str[:8].isin(runs["run"].astype(str))
flags = flags[flags_mask]
scalars_mask = scalars["run"].str[:8].isin(runs["run"].astype(str))
scalars = scalars[scalars_mask]

In [ ]:
imputer_mapping = ["simple", "knn", "mice", "mida", "dae_mvec", "vae_ifac", "ap_new"]
imputer_order = ["Simple", "KNN", "MICE", "MIDA", "DAE", "VAE", "APnew"]
mechanism_order = ["MCAR", "MAR", "MNAR"]
# percent_order = ["0.33", "0.66"]
percent_order = [33.0, 66.0]
predictor_mapping = ["logistic_regression", "random_forest"]
predictor_order = ["Logistic Regression", "Random Forest"]
# predict_metric_order = ["TN", "FP", "TP", "FN", "Brier-score", "F1-score", "Precision-score", "Recall-score", "PR-AUC", "ROC-AUC"] # If colwrap=2
# predict_metric_mapping = ["TN", "FP", "TP", "FN", "Brier-score", "F1-score", "Precision-score", "Recall-score", "PR-AUC", "ROC-AUC"] # If colwrap=2
# predict_metric_order = ["True Negative", "False Positive", "True Positive", "False Negative", "Brier-score", "F1-score", "Precision-score", "Recall-score", "PR-AUC", "ROC-AUC"] # If colwrap=2
# ignores F1 score
predict_metric_mapping = ["TN", "FP", "TP", "FN", "Brier-score", "Precision-score", "Recall-score", "PR-AUC", "ROC-AUC"] 
predict_metric_order = ["True Negative", "False Positive", "True Positive", "False Negative", "Brier-score", "Precision-score", "Recall-score", "PR-AUC", "ROC-AUC"] 
name_mapping = {
        "Method": dict(zip(imputer_mapping, imputer_order)),
        "Predictive Model": dict(zip(predictor_mapping, predictor_order)),
        "Imputation Metric": {"RMSE-missingonly": "RMSE", "MAAPE-missingonly": "MAAPE", "AccuracyPerBin-missingonly": "Accuracy Over Bins"},
        "Metric": dict(zip(predict_metric_mapping, predict_metric_order))
    }

In [ ]:
runs.head()

# Impute metrics

In [ ]:
from IPython.display import display

test_mask = scalars["tag"].str.startswith("impute/test-")
fully_observed_mask = scalars["prefix"].str.startswith("F.O.")
values = scalars[test_mask & fully_observed_mask]
values[["percent", "mech", "method"]] = values["prefix"].str.split("/", expand=True).drop(0, axis=1)
# get the metric name (everything after impute/test-)
values["metric"] = values["tag"].str[len("impute/test-"):]
table = pd.DataFrame()
table[["Imputation Metric", "Percent", "Mechanism", "Method", "first_val"]] = values[["metric", "percent", "mech", "method", "first_val"]]

# Filter for missing only
table = table[table["Imputation Metric"].str.endswith("missingonly")]

# Rename for figure
for col_name, col_mapping in name_mapping.items():
    if col_name in table:
        table[col_name] = table[col_name].map(col_mapping)
        

table["Percent"] = table["Percent"].astype(float)*100


def impute_table_to_latex(table: pd.DataFrame) -> str:
    table = table[table["Imputation Metric"] != "Accuracy Over Bins"]
    table = table.pivot_table(index=["Imputation Metric", "Mechanism", "Percent"], columns="Method", values="first_val") 
    table = table[imputer_order]
    display(table)
    latex = table.to_latex(float_format="%.3f")
#     print(latex)
    return latex
impute_table_to_latex(table)

In [ ]:
# with sns.plotting_context(font_scale=1.5):
table_without_binaccuracy = table[table["Imputation Metric"] != "Accuracy Over Bins"]
g = sns.catplot(x="Method", y="first_val", hue="Percent", col="Mechanism", row="Imputation Metric", data=table_without_binaccuracy,
                ci=None, sharey="row", legend=False,
                hue_order=percent_order, order=imputer_order, col_order=mechanism_order,
                markers=["o", "x"], join=False, #linestyles=["-", "--"], 
                margin_titles=True, kind="point")
g.add_legend(loc="upper right", bbox_to_anchor=(0.5, 1.10), ncol=2, title="Percent Missing", frameon=True)
g.set_titles(col_template="{col_name}", row_template="{row_name}").set_xticklabels(rotation=45).set_axis_labels("Imputation Method", "").tight_layout()
plt.suptitle("Imputation Metrics", x= 0.39, y=1.135, weight="bold")

In [ ]:
#heatmap version
# from matplotlib import pyplot
# tmp = table_without_binaccuracy.pivot_table(index=["Method"], columns=["Percent", "Imputation Metric", "Mechanism"], values="first_val")
# fig, ax = pyplot.subplots(figsize=(20,5))
# g = sns.heatmap(tmp, yticklabels=imputer_order, annot=True, ax=ax)
# # tmp.unstack()

def draw_heatmap(*args, **kwargs):
    data = kwargs.pop('data')
    # d = data.pivot_table(index=["Method"], columns=["Percent"], values="first_val")
    d = data.pivot_table(index=["Percent"], columns=["Method"], values="first_val")
    sns.heatmap(d, annot=True, **kwargs)

g = sns.FacetGrid(table_without_binaccuracy, col="Mechanism", row="Imputation Metric", col_order=mechanism_order,
                    margin_titles=True, height=3, aspect=3)
# g.map(sns.heatmap, "first_val", annot=True)
g.map_dataframe(draw_heatmap)
g.set_titles(col_template="{col_name}", row_template="{row_name}").set_xticklabels(rotation=45).set_axis_labels("Percent Missing", "").tight_layout()
plt.suptitle("Imputation Metrics", x= 0.5, y=1.05, weight="bold")


In [ ]:
# table[table["Imputation Metric"] == "Accuracy Over Bins"]
g = sns.barplot(x="Mechanism", y="first_val", hue="Percent", data=table[table["Imputation Metric"] == "Accuracy Over Bins"],
                order=mechanism_order)
# dir(g)
# set hatches for alternate to visually distinguish aside from color
hatches = [None, "///"]
j = -1
# will iterate over each colour at a time from left to right, so it will iterate over the left blue bar, then middle, then right, then the left orange bar, etc.
for i, bar in enumerate(g.patches):
    # every number columns change the hatch pattern
    if  i % len(mechanism_order) == 0:
        j += 1
    bar.set_hatch(hatches[j])

g.set_ylabel("")
plt.suptitle("Accuracy Over Bins", y=1.25)
plt.legend(loc="upper center", bbox_to_anchor=(0.5, 1.40), ncol=2, title="Percent Missing")

def binacc_table_to_latex(table: pd.DataFrame) -> str:
    table = table[table["Imputation Metric"] == "Accuracy Over Bins"]
    table = table.pivot_table(index=["Mechanism", "Percent"], values="first_val") 
    display(table)
    latex = table.to_latex(float_format="%.3f")
    print(latex)
binacc_table_to_latex(table)

# Prediction Task Performance

# Entire Dataset

In [ ]:
predict_mask = scalars["tag"].str.startswith("predict/")
full_mask = scalars["prefix"].str.startswith("full")
res = scalars[predict_mask & full_mask]
# if prefix == "F.O.":  # filter for fully observed
#     res = res[res["prefix"].str.startswith("F.O.")]
#     res[["data", "percent", "mech", "method", "model"]] = res["prefix"].str.split("/", expand=True)

res[["data", "method", "model"]] = res["prefix"].str.split("/", expand=True)

res["metric"] = res["tag"].apply(lambda logname: logname.split("/")[-1])

aggregate_mask = scalars["tag"].str.startswith("predict-aggregate/") 
full_mask = scalars["prefix"].str.startswith("full")
lower_mask = scalars["tag"].str.endswith("-lower") 
upper_mask = scalars["tag"].str.endswith("-upper") 
err_lower = scalars[aggregate_mask & lower_mask & full_mask]["first_val"]
err_upper = scalars[aggregate_mask & upper_mask & full_mask]["first_val"]
err_lower.index = res.index
err_upper.index = res.index
res["err_lower"] = err_lower
res["err_upper"] = err_upper
# err_upper["first_val"].subtract(err_lower["first_val"], fill_value=0)

# validation: checking that the order is exactly the same so i don't have to worry
# pd.concat([res["prefix"].reset_index(drop=True), err_lower["prefix"].reset_index(drop=True), err_upper["prefix"].reset_index(drop=True)],axis=1)

res = res[["metric", "method", "model", "avg_val", "err_lower", "err_upper"]]
res.rename(columns={"metric": "Metric", "method": "Method", "model": "Predictive Model"}, inplace=True)
# Rename for figure
for col_name, col_mapping in name_mapping.items():
    if col_name in res:
        res[col_name] = res[col_name].map(col_mapping)

def pred_table_to_latex(res: pd.DataFrame) -> str:
    res = res.pivot_table(index=["Metric", "Predictive Model"], columns="Method", values="avg_val") 
    res = res[imputer_order]
    display(res)
    latex = res.to_latex(float_format="%.3f")
#     print(latex)
    return latex
pred_table_to_latex(res)

In [ ]:

g = sns.catplot(x="Method", y="avg_val", hue="Predictive Model", col="Metric", data=res,
                order=imputer_order, col_order=predict_metric_order, hue_order=predictor_order,
                markers=["o", "x"], join=False, # linestyles=["-", "--"], 
                sharey=False, col_wrap=3, aspect=1.3,
                dodge=0.2, kind="point",
                legend=False)
# g.map(plt.errorbar, x="method", y="avg_val", hue="model", col="metric", yerr=["err_lower", "err_upper"])

def plot_errbar(x: str, y: str, hue: str, data: pd.DataFrame, order, **kwargs):
    """Plots error bar into each facet in the facetgrid from seaborn with dodge + color per hue."""
    # get everything pivoted to have a column per hue, and then sort index
    err_lower = data.pivot(index=x, columns=hue, values="err_lower").T[order].T
    err_upper = data.pivot(index=x, columns=hue, values="err_upper").T[order].T
    median = data.pivot(index=x, columns=hue, values=y).T[order].T
    err_lower = median - err_lower
    err_upper = err_upper - median

    ax = plt.gca()
    colors = sns.color_palette()
    for i, hue in enumerate(err_lower):
        dodge = 6*i if i != 0 else -5
        trans = ax.transData + transforms.ScaledTranslation(dodge/72., 0, ax.figure.dpi_scale_trans)
        ax.errorbar(x=median[hue].index, y=median[hue], yerr=[err_lower[hue].values, err_upper[hue].values],
                    ecolor=colors[i], fmt='none', transform=trans, **kwargs)
g.map_dataframe(plot_errbar, x="Method", y="avg_val", hue="Predictive Model", order=imputer_order)


# place boxes upper right hand corner at x,y for bbox
g.add_legend(loc="upper right", bbox_to_anchor=(0.5, 1.07), ncol=2, title="Predictive Model", frameon=True)
g.set_titles(col_template="{col_name}").set_xticklabels(rotation=45).set_axis_labels("Imputation Method", "").tight_layout()
plt.suptitle("Prediction Metrics", x=0.35, y=1.09, weight="bold")

In [ ]:
# HeatMap Version
ignore_metrics = ["True Negative", "True Positive", "False Positive", "False Negative"]
a = res[res["Predictive Model"] == "Logistic Regression"].pivot_table(index=["Metric", "Method"], values="avg_val")
# ignore the large scale ones
a = a[~a.index.get_level_values("Metric").isin(ignore_metrics)]
a = a.unstack()

# g = sns.heatmap(a.unstack()*100, annot=a)
# a.apply(lambda : a.quantile(row), axis=1, result_type="expand")
a.quantile(np.linspace(.1, 1, 9, 0))
# LAID TO REST: the different metrics are on different scales :( I would only be able to heatmap some of them)

# Time

In [ ]:
# help(runs.iloc[0]["run"].value)
# keep = runs["operation"] == "imputer"
# timings = runs[keep].compare()
# timings[["time", "method", "percent-missing", "missingness-mechanism"]].pivot_table(index=["percent-missing", "missingness-mechanism"], columns="method", values="time")
# convert string time to integer second
grouped_timings = runs[["method", "fully-observed", "predictors"]]
grouped_timings["time"] = runs["time"].dt.seconds
ax = sns.pointplot(data=grouped_timings, x="method", y="time", hue="fully-observed")
ax.set_xlabel("Imputation Method")
ax.set_ylabel("Time (seconds)")